# TensorRT - Installation

Start with an understanding of your model and the TensorFlow versions - which leads to the CUDA versions.   If on SageMaker - this will not be super flexible.   AWS may be lagging on TF version.

IMPORTANT NOTE 
- use deb (dpkg) on Ubuntu
- use rpm (yum) on AWS Linux

I learned the hardway - if you use the *.run files, which looks easier - it may not register the packages correctly

### READ THE DOCUMENTATION CAREFULLY
https://docs.nvidia.com/deeplearning/sdk/tensorrt-install-guide/index.html
TensorFlow 1.14 required  
CUDA 10.0 or 10.2 required -- 10.1 NOT supported (and just guess what version we have!)
So problem - how do you get SageMaker to use CUDA 10.0 an the 410 driver?  


### TensorRT version 7.0 
### CUDA 10.0 w/ driver 410
Instructions dictate TensorFlow 1.14  & CUDA 10.1 is NOT supported.   There are good instructions on CFA Confluence for installing CUDA 10.0, cuDNN and the 410 driver.   Go there if you need help.

## Using a physical computer
e.g.   REV Center - Inspiron, tag: DTK2*
    
Use SageMaker if you can to avoid this hassle - but... I'm stuck so I'm back to a computer.   Here are some things you may have to deal with.   You need to do this before you can get started -- even with this notebook.

#### update conda
RUN these from terminal - don't run them here.   I'm just documenting here  
`conda update -n base -c defaults conda`  

My Anaconda was totally hosed - I had re-install
`rm -rf ~/anaconda3`  
`bash ./Anaconda3-2019.10-Linux-x86_64.sh -u`  

Add your environments (all with Python 3.7)  
Using anaconda-navigator GUI - I added:  
tensorRT - note, I used TF 1.14 because SageMaker uses 1.14.  TensorRT wants a frozen graph.   
20200113 - Instructuions say "TensorFlow export requires 1.14.0"  

`source activate tensorRT`  
`pip instal tensorflow-gpu==1.14`  
`python -m ipykernel install --user --name tensorRT --display-name "Python (tensorRT)"`  

THIS NOTEBOOK SHOULD BE RUNNING ON tensorRT environment (so this is kind of a catch-22)

### Back to General Instructions
Regardless of SageMaker or Physical Computer

In [ ]:
# this will show you your CUDA version and driver version
# note that nvcc is NOT installed

# the symbol, cuda, is pointing to cuda-10.0
! ls /usr/local -la

# BUT, nvidia-smi says 10.1
! nvidia-smi

# I learned the hardway, that means CUDA 10.1 is installed - so install TensorRT for 10.1
# you can ignore the symbol (cuda) for now

### verify you have cuDNN installed
- hmm, seems this is difficult

#### 1.  NVIDIA says to download the samples then make the mnist file and test it
that is several steps - but it is straight forward

#### 2.  $ locate - do this from terminal 
`sudo updatedb`  
`locate libcudnn`  

#### 3.  this method seems to be obsolete - it will give you a false negative
`cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2`  

#### 4.  after installing 7.6.5, this would work
`ls /usr/lib/x86_64-linux-gnu/libcudnn.so.7.6.5`  

In [ ]:
# we shouldn't do anything that requires tensorflow - but let's check it
import tensorflow as tf
print (tf.__version__)

In [ ]:
# verify cuBLAS (part of the CUDA toolkit) is installed
! cat /usr/local/cuda/include/cublas.h | grep CUBLAS

### PyCUDA
Instructions say you need PyCUDA - which goes on to say, get your CUDA & Driver installed first, then install PyCUDA.   And if you have problems, you may have to compile it yourself  :(  

From your tensorRT environment (tensorRT) ... $  
` pip install 'pycuda>=2019.1.1'`

### 20200113 - on SageMaker p2
Tensorflow 1.14.0  
CUDA 10.1  
NVIDIA Driver 418.87  

So we 


In [ ]:
import os

In [ ]:
# Globals

PROJECT_DIR = os.getcwd()
BUCKET = 's3://cfaanalyticsresearch-sagemaker'

SAGEMAKER_DIR = os.path.join(PROJECT_DIR, "..")


## Get the TensorRT software

https://developer.nvidia.com/nvidia-tensorrt-7x-download
### Installation Instructions
https://docs.nvidia.com/deeplearning/sdk/tensorrt-archived/tensorrt-700/tensorrt-install-guide/index.html  
   

928 MB This will not go well if you are on CFA network !!!!
  - leverage a Workspace if possible (super fast)
  - use a curl command (I didn't figure this out) - downloading directly to SageMaker should be fast
  - use S3 (I put current packages on the cfaanalyticsresearch-sagemaker/software
  - Don't download over CFA/COWS to your computer then upload!! it will fail
For SageMaker:  get a RPM package (20200113) CentOS/RedHat 7 CUDA 10.0  
For Ubuntu: get the deb package

In [ ]:
# SageMaker
# - upload the RPM package to the SageMaker directory; 

# look in S3
! aws s3 ls {BUCKET}/software/

# download to SageMaker directory
# - you only have to do this once
# - but you'll need to install every time you reboot

# TensorRT
! aws s3 cp {BUCKET}/software/ {SAGEMAKER_DIR} --exclude "*.*" --include "nv-tensorrt-repo-rhel7-cuda10.0-trt7*.rpm" --recursive

In [ ]:
# Physical Computer
# - I pulled the image off the USB
# put it in ~/Downloads
# DON'T FORGET - on the laptop (which is Ubuntu), you want the Ubuntu version
# nv-tensorrt-repo-ubuntu1804-cuda10.0*.deb

### SageMaker Installation
Read 4.2 for RPM installation  
Jupyter won't keep the variable names correctly so,  cut paste this into the terminal.  
- open a Jupyter terminal
- go to SageMaker directory

c
Pre-requisites
Error: Package: libnvinfer7-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cublas-10-0
Error: Package: libnvinfer-devel-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cublas-dev-10-0
Error: Package: libnvinfer-devel-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cudart-dev-10-0
Error: Package: libnvinfer7-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cudart-10-0
           
`wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda_10.2.89_440.33.01_linux.run`
`sudo sh cuda_10.2.89_440.33.01_linux.run`         


CUDA Toolkit (to get cuBLAS-10.0)
https://developer.nvidia.com/cuda-10.0-download-archive  
I put 10.1 on S3 and you downloaded it above - so you just have to install.  
`sudo rpm -i cuda-repo-rhel7-10-1-local-10.1.105-418.39-1.0-1.x86_64.rpm`  
`sudo yum clean all`  
`sudo yum install cuda`  

Get the tag value from the current version (file) you downloaded 

`tag="cuda10.0-trt7.0.0.11-ga-20191216"`  
`sudo rpm -Uvh nv-tensorrt-repo-rhel7-${tag}-1-1.x86_64.rpm`  
`sudo yum clean expire-cache`  

### Ubuntu Installation
WARNING - be VERY careful with the punctuation here - it MATTERS!  ticks, right ticks, quotes - etc  
`tag="cuda10.0-trt7.0.0.11-ga-20191216"`  
`sudo dpkg -i nv-tensorrt-repo-${os}-${tag}_1-1_amd64.deb`  
`sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub` 

`sudo apt-get update`  
`sudo apt-get install tensorrt`

`sudo apt-get install python3-libnvinfer-dev`  
`sudo apt-get install uff-converter-tf`  

`dpkg -l | grep TensorRT`  
